# Cryptocurrency Price Prediction

In [1]:
# pip install yfinance

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime
from datetime import date, timedelta

today = date.today()

d1 = today.strftime('%Y-%m-%d')
end_date = d1

d2 = date.today() - timedelta(days=1000)
d2 = d2.strftime('%Y-%m-%d')
start_date = d2

# print(start_date)
data = yf.download('BTC-USD',
                   start=start_date,
                   end=end_date,
                   progress=False,
                   auto_adjust=False)

# data = data.reset_index()
# print(data)

# data = [['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
# data.columns = ['date', 'open', 'high', 'low', 'close', 'adj close', 'volume']
# print(data)

data['Date'] = data.index

data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
data.reset_index(drop=True, inplace=True)
data.columns = data.columns.get_level_values(0)
print(data.head(n=1))


# data.to_csv('dataset/new_dataset.csv', index=False)
# dataframe = pd.read_csv('dataset/new_dataset.csv')

Price       Date          Open          High           Low         Close  \
0     2022-12-27  16919.291016  16959.845703  16642.072266  16717.173828   

Price     Adj Close       Volume  
0      16717.173828  15748580239  


In [3]:
print(data.head())

Price       Date          Open          High           Low         Close  \
0     2022-12-27  16919.291016  16959.845703  16642.072266  16717.173828   
1     2022-12-28  16716.400391  16768.169922  16497.556641  16552.572266   
2     2022-12-29  16552.322266  16651.755859  16508.683594  16642.341797   
3     2022-12-30  16641.330078  16643.427734  16408.474609  16602.585938   
4     2022-12-31  16603.673828  16628.986328  16517.519531  16547.496094   

Price     Adj Close       Volume  
0      16717.173828  15748580239  
1      16552.572266  17005713920  
2      16642.341797  14472237479  
3      16602.585938  15929162910  
4      16547.496094  11239186456  


In [4]:
print(data.tail())

Price       Date           Open           High            Low          Close  \
995   2025-09-17  116840.507812  117328.609375  114794.976562  116468.507812   
996   2025-09-18  116461.265625  117911.789062  116188.796875  117137.203125   
997   2025-09-19  117137.671875  117479.757812  115141.820312  115688.859375   
998   2025-09-20  115691.125000  116191.148438  115473.523438  115721.960938   
999   2025-09-21  115730.226562  115901.085938  115252.578125  115306.093750   

Price      Adj Close       Volume  
995    116468.507812  60528025996  
996    117137.203125  49457272032  
997    115688.859375  38828473971  
998    115721.960938  22864449614  
999    115306.093750  22495852193  


In [5]:
data.shape

(1000, 7)

In [6]:
import plotly.graph_objects as pg
from plotly.offline import iplot
figure = pg.Figure(data=[pg.Candlestick(x=data['Date'],
                                        open=data['Open'],
                                        high=data['High'],
                                        low=data['Low'],
                                        close=data['Close'])])

# figure.update_layout(title = 'Bitcoin Price Analysis',
#                      xaxis_rangeslider_visible = False)
figure.show()
# iplot(figure)

In [7]:
# df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')

# print(df.head())
# print(df.tail())
# print(df['AAPL.Open'])
# print(df.shape)

# fig = pg.Figure(data=[pg.Candlestick(x=df['Date'],
#                 open=df['AAPL.Open'],
#                 high=df['AAPL.High'],
#                 low=df['AAPL.Low'],
#                 close=df['AAPL.Close'])])

# fig.show()

In [8]:
correlation = data.corr()
print(correlation['Close'].sort_values(ascending=False))

Price
Close        1.000000
Adj Close    1.000000
High         0.999462
Low          0.999269
Open         0.998664
Date         0.960467
Volume       0.669440
Name: Close, dtype: float64


In [ ]:
# pip install autots

Note: you may need to restart the kernel to use updated packages.


## Prediction the model

In [10]:
from autots import AutoTS
model = AutoTS(forecast_length=30, frequency='infer', ensemble='simple')
model = model.fit(data, date_col='Date', value_col='Close', id_col=None)

prediction = model.predict()
forecast = prediction.forecast
print(forecast)

Using 2 cpus for n_jobs.
Data frequency is: D, used frequency is: D
Model Number: 1 with model AverageValueNaive in generation 0 of 25
Model Number: 2 with model AverageValueNaive in generation 0 of 25
Model Number: 3 with model AverageValueNaive in generation 0 of 25
Model Number: 4 with model DatepartRegression in generation 0 of 25
Model Number: 5 with model DatepartRegression in generation 0 of 25
Model Number: 6 with model DatepartRegression in generation 0 of 25


/run/media/ooka/cae585b1-9ae2-453d-9643-e7f830589189/jinx/Belajar/ML Projects/.venv/lib/python3.13/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

/run/media/ooka/cae585b1-9ae2-453d-9643-e7f830589189/jinx/Belajar/ML Projects/.venv/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:602: ConvergenceWarning:

lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



Model Number: 7 with model DatepartRegression in generation 0 of 25
Template Eval Error: ImportError('Tensorflow not available, install with pip install tensorflow.') in model 7 in generation 0: DatepartRegression
Model Number: 8 with model ETS in generation 0 of 25
Model Number: 9 with model ETS in generation 0 of 25
Model Number: 10 with model GLM in generation 0 of 25
Model Number: 11 with model GLM in generation 0 of 25
Model Number: 12 with model GLS in generation 0 of 25
Model Number: 13 with model GLS in generation 0 of 25
Model Number: 14 with model LastValueNaive in generation 0 of 25
Model Number: 15 with model LastValueNaive in generation 0 of 25
Model Number: 16 with model LastValueNaive in generation 0 of 25
Model Number: 17 with model LastValueNaive in generation 0 of 25
Model Number: 18 with model SeasonalNaive in generation 0 of 25
Model Number: 19 with model SeasonalNaive in generation 0 of 25
Model Number: 20 with model SeasonalNaive in generation 0 of 25
Model Number

/run/media/ooka/cae585b1-9ae2-453d-9643-e7f830589189/jinx/Belajar/ML Projects/.venv/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:602: ConvergenceWarning:

lbfgs failed to converge after 250 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=250).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



Model Number: 28 with model DatepartRegression in generation 0 of 25
Model Number: 29 with model ETS in generation 0 of 25
Model Number: 30 with model ARDL in generation 0 of 25
Model Number: 31 with model UnivariateMotif in generation 0 of 25
Model Number: 32 with model UnivariateMotif in generation 0 of 25
Model Number: 33 with model SectionalMotif in generation 0 of 25
Model Number: 34 with model SectionalMotif in generation 0 of 25
Model Number: 35 with model FBProphet in generation 0 of 25
Template Eval Error: ModuleNotFoundError("No module named 'fbprophet'") in model 35 in generation 0: FBProphet
Model Number: 36 with model SeasonalNaive in generation 0 of 25
Model Number: 37 with model DatepartRegression in generation 0 of 25
Model Number: 38 with model ARCH in generation 0 of 25
Template Eval Error: ImportError('`arch` package must be installed from pip') in model 38 in generation 0: ARCH
Model Number: 39 with model Cassandra in generation 0 of 25
Model Number: 40 with model S

...

37 - Ensemble with avg smape 1.83: 
38 - Ensemble with avg smape 1.77: 
39 - Ensemble with avg smape 1.87: 
40 - Ensemble with avg smape 2.14: 
41 - Ensemble with avg smape 2.17: 
42 - Ensemble with avg smape 121.48: 
Template Eval Error: Exception("Transformer HistoricValues failed on fit from params fake_date {'0': {'rows': 1, 'lag': 1, 'method': 'additive', 'strength': 1.0, 'first_value_only': False, 'threshold': 10, 'threshold_method': 'mean'}, '1': {'window': 28}} with error KeyError('2')") in model 42 in generation 0: Ensemble
Model Number: 43 of 9 with model Ensemble for Validation 3
43 - Ensemble with avg smape 2.05: 
44 - Ensemble with avg smape 1.93: 
45 - Ensemble with avg smape 2.09: 
46 - Ensemble with avg smape 2.44: 
47 - Ensemble with avg smape 2.16: 
48 - Ensemble with avg smape 121.48: 
Template Eval Error: Exception("Transformer HistoricValues failed on fit from params fake_date {'0': {'rows': 1, 'lag': 1, 'method': 'additive', 'strength': 1.0, 'first_value_only': Fa

/run/media/ooka/cae585b1-9ae2-453d-9643-e7f830589189/jinx/Belajar/ML Projects/.venv/lib/python3.13/site-packages/statsmodels/tsa/filters/hp_filter.py:100: SparseEfficiencyWarning:

spsolve requires A be CSC or CSR matrix format

/run/media/ooka/cae585b1-9ae2-453d-9643-e7f830589189/jinx/Belajar/ML Projects/.venv/lib/python3.13/site-packages/statsmodels/tsa/filters/hp_filter.py:100: SparseEfficiencyWarning:

spsolve requires A be CSC or CSR matrix format



49 - Ensemble with avg smape 2.03: 
50 - Ensemble with avg smape 1.92: 
51 - Ensemble with avg smape 2.18: 
52 - Ensemble with avg smape 2.53: 
53 - Ensemble with avg smape 2.16: 
54 - Ensemble with avg smape 121.48: 
Template Eval Error: Exception("Transformer HistoricValues failed on fit from params fake_date {'0': {'rows': 1, 'lag': 1, 'method': 'additive', 'strength': 1.0, 'first_value_only': False, 'threshold': 10, 'threshold_method': 'mean'}, '1': {'window': 28}} with error KeyError('2')") in model 54 in generation 0: Ensemble


/run/media/ooka/cae585b1-9ae2-453d-9643-e7f830589189/jinx/Belajar/ML Projects/.venv/lib/python3.13/site-packages/autots/evaluator/auto_model.py:3068: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/run/media/ooka/cae585b1-9ae2-453d-9643-e7f830589189/jinx/Belajar/ML Projects/.venv/lib/python3.13/site-packages/autots/evaluator/auto_model.py:3068: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



Price               Close
2025-09-22  115306.093750
2025-09-23  116217.906474
2025-09-24  116455.324421
2025-09-25  116356.167891
2025-09-26  116210.750679
2025-09-27  116394.505918
2025-09-28  116216.785606
2025-09-29  116022.911983
2025-09-30  115732.811490
2025-10-01  115711.347013
2025-10-02  115562.626879
2025-10-03  115522.162374
2025-10-04  115072.210821
2025-10-05  115254.632939
2025-10-06  115411.190781
2025-10-07  115610.355134
2025-10-08  116026.423794
2025-10-09  116505.996535
2025-10-10  116796.353300
2025-10-11  116967.479513
2025-10-12  117289.683838
2025-10-13  118018.469627
2025-10-14  118186.897159
2025-10-15  118957.804965
2025-10-16  118927.762826
2025-10-17  119576.862023
2025-10-18  119977.436609
2025-10-19  120559.487967
2025-10-20  121052.297169
2025-10-21  121805.341442


## Visualising the result|